In [ ]:
import torch
from train_peclet_model import PecletModelTrainer
from ThreeLayerCNNRegressor import ThreeLayerCNNRegressor, JumboThreeLayerCNNRegressor
import json

from config import MODEL_STATS_PATH, DB_PATH, MODEL_DEM_PATH, LABEL_QUERY, OUTPUTS_TABLE, WEIGHTS_PATH, NN_SEEDS


In [ ]:
with open(MODEL_STATS_PATH, 'r') as f:
    stats = json.load(f)
retrain = False
data_types = [{'type': 'dem',
               'data_path': MODEL_DEM_PATH},
              {'type': 'accumulation',
               'data_path': MODEL_ACC_PATH}
               ]


In [ ]:
for seed in NN_SEEDS:
    for data in data_types:
        torch.manual_seed(seed)
        weights_path = os.path.join(WEIGHTS_DIR, f"{data['type']_{seed}_weights.pt")
        trainer = PecletModelTrainer(DB_PATH,
                                    data['data_path'],
                                    ThreeLayerCNNRegressor,
                                    f"SELECT {LABEL_QUERY} FROM {OUTPUTS_TABLE}",
                                    epochs = 50,
                                    learning_rate=0.0001,
                                    **stats[data['type']],
                                    **stats['labels'])
        trainer.load_weights(weights_path)
        trainer.evaluate()
        csv_path = os.path.join(RESULTS_DIR, f"{data['type']}_{seed}_results.csv")
        trainer.test_df.to_csv(csv_path)


We also train a "jumbo" model with larger layers.

In [ ]:
weights_path = os.path.join(WEIGHTS_dir, 'dem_jumbo_run_weights.pt')
torch.manual_seed(NN_SEEDS[0])
traier = PecletModelTrainer(DB_PATH,
                            MODEL_DEM_PATH,
                            JumboThreeLayerCNNRegressor,
                            f"SELECT {LABEL_QUERY} FROM {OUTPUTS_TABLE}",
                            epochs = 50,
                            learning_rate=0.0001,
                            **stats['dem'],
                            **stats['labels'])
trainer.load_weights(weights_path)
trainer.evaluate()
csv_path = os.path.join(RESULTS_DIR, 'dem_jumbo_run_results.pt')
trainer.test_df.to_csv(csv_path)
